In [1]:
from MALBuddy import MALBuddy
import pandas as pd
import os
import re
import numpy as np

client_info_filepath = "client_info.json"
token_filepath = "token.json"

In [2]:
buddy = MALBuddy(client_info_filepath, token_filepath)

In [3]:
user = 'chocopieeater'
anime_list = buddy.get_anime_list(user)
anime_list.head()

,id,title,main_picture,status,score,num_episodes_watched,is_rewatching,updated_at,finish_date,start_date
0,41457,86,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,0,False,2021-05-02T00:36:51+00:00,NaN,NaN
1,22199,Akame ga Kill!,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,8,24,False,2019-02-26T07:05:44+00:00,NaN,NaN
2,41433,Akudama Drive,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,7,False,2020-12-28T21:14:38+00:00,NaN,NaN
3,22729,Aldnoah.Zero,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,6,12,False,2018-08-07T02:05:12+00:00,NaN,NaN
4,27655,Aldnoah.Zero 2nd Season,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,11,False,2018-08-07T01:31:22+00:00,NaN,NaN


In [4]:
anime_list

,id,title,main_picture,status,score,num_episodes_watched,is_rewatching,updated_at,finish_date,start_date
0,41457,86,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,0,False,2021-05-02T00:36:51+00:00,NaN,NaN
1,22199,Akame ga Kill!,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,8,24,False,2019-02-26T07:05:44+00:00,NaN,NaN
2,41433,Akudama Drive,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,7,False,2020-12-28T21:14:38+00:00,NaN,NaN
3,22729,Aldnoah.Zero,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,6,12,False,2018-08-07T02:05:12+00:00,NaN,NaN
4,27655,Aldnoah.Zero 2nd Season,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,11,False,2018-08-07T01:31:22+00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
211,39617,Yakusoku no Neverland 2nd Season,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,4,8,False,2021-06-05T05:50:26+00:00,NaN,NaN
212,32615,Youjo Senki,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,8,12,False,2019-04-18T07:49:11+00:00,NaN,NaN
213,49233,Youjo Senki II,{'medium': 'https://api-cdn.myanimelist.net/im...,plan_to_watch,0,0,False,2021-06-19T16:47:11+00:00,NaN,NaN
214,37055,Youjo Senki Movie,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,8,1,False,2019-11-10T03:49:38+00:00,2019-11-09,2019-11-09


In [5]:
buddy.generate_and_write_ratings('41457', 'title',num_pages=5)

Successfully loaded pages: : 1, 2, 3, 4, 5, 

KeyboardInterrupt: 

In [6]:
id_title = zip(anime_list['id'].to_numpy(),anime_list['title'].to_numpy())
for id_, title in id_title:
    buddy.generate_and_write_ratings(id_, title, num_pages=5)
    print()

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\86.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\akame_ga_kill.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\akudama_drive.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\aldnoahzero.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\aldnoahzero_2nd_season.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\angel_beats.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\ansatsu_kyoushitsu.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\ansatsu_kyoushitsu_2nd_season.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\ao_no_exorcist.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\assassins_pride.json 

Successfully

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\hataraku_saibou_tv.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\hinamatsuri_tv.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\horimiya.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\houseki_no_kuni_tv.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\howl_no_ugoku_shiro.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\hunter_x_hunter_2011.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\hyakuren_no_haou_to_seiyaku_no_valkyria.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\ijiranaide_nagatorosan.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\isekai_maou_to_shoukan_shoujo_no_dorei_majutsu.json 

Successfully loaded pages: : 1, 2, 3, 4,

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\made_in_abyss.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\made_in_abyss_movie_1_tabidachi_no_yoake.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\made_in_abyss_movie_3_fukaki_tamashii_no_reimei.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\made_in_abyss_retsujitsu_no_ougonkyou.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\magi_sinbad_no_bouken_tv.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\magi_the_kingdom_of_magic.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\magi_the_labyrinth_of_magic.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\mahouka_koukou_no_rettousei.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\mahouka_kouk

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\princess_connect_redive.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\psychopass.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\rezero_kara_hajimeru_isekai_seikatsu.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\rezero_kara_hajimeru_isekai_seikatsu__memory_snow.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\rezero_kara_hajimeru_isekai_seikatsu_2nd_season.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\rezero_kara_hajimeru_isekai_seikatsu_2nd_season_part_2.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\rokka_no_yuusha.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\seirei_gensouki.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\sei

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\tensura_nikki_tensei_shitara_slime_datta_ken.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\toaru_majutsu_no_index.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\tokyo_ghoul.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\tokyo_ghoul_a.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\tokyo_ghoulre.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\tokyo_ghoulre_2nd_season.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\tsuujou_kougeki_ga_zentai_kougeki_de_nikai_kougeki_no_okaasan_wa_suki_desu_ka.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\uzakichan_wa_asobitai.json 

Successfully loaded pages: : 1, 2, 3, 4, 5, 
Ratings saved to anime_ratings\vanitas_no_karte.json 

Suc